In [ ]:
import os
import kagglehub

# --- Définition des chemins ---
# Le dossier principal qui contiendra les données
DATA_DIR = "data"
# Le dossier de destination spécifique au téléchargement Kaggle
CSV_DIR = os.path.join(DATA_DIR, "csv")
# Le handle Kaggle pour le dataset
KAGGLE_HANDLE = "afrniomelo/tep-csv"

# --- 1. Vérification et Création des Dossiers ---
if not os.path.exists(DATA_DIR):
    # !mkdir data
    os.makedirs(DATA_DIR, exist_ok=True)
    print(f"Dossier créé: {DATA_DIR}")

if not os.path.exists(CSV_DIR):
    # !mkdir data/csv
    os.makedirs(CSV_DIR, exist_ok=True)
    print(f"Dossier créé: {CSV_DIR}")


# --- 2. Vérification du Contenu avant le Téléchargement ---
# On vérifie si le dossier de destination contient déjà des fichiers (autres que les fichiers cachés/système)
existing_files = [f for f in os.listdir(CSV_DIR) if not f.startswith('.')]

if existing_files:
    print(f"✅ Le dossier {CSV_DIR} contient déjà {len(existing_files)} fichier(s).")
    print("Le téléchargement sera ignoré.")

    # Optional: Si vous devez obtenir le chemin d'accès au dossier pour la suite du script,
    # vous pouvez le définir ici.
    path = CSV_DIR

else:
    print(f"❌ Le dossier {CSV_DIR} est vide. Démarrage du téléchargement...")

    # Download latest version
    # Note: L'argument 'path' de kagglehub.dataset_download DOIT être le chemin
    # où le dataset sera téléchargé (CSV_DIR dans ce cas)
    try:
        path = kagglehub.dataset_download(handle=KAGGLE_HANDLE, path=CSV_DIR)
        print("Téléchargement réussi!")
    except Exception as e:
        print(f"Erreur lors du téléchargement Kaggle : {e}")
        path = None


# --- 3. Affichage du Résultat ---
if path:
    print("Chemin d'accès aux fichiers du dataset:", os.path.abspath(path))

✅ Le dossier data/csv contient déjà 8 fichier(s).
Le téléchargement sera ignoré.
Chemin d'accès aux fichiers du dataset: /home/bapt/code/Monitor-the-Reactor/Explo/data/csv


In [ ]:
import pandas as pd
import os
from typing import Iterator, Generator, List, Dict

# --- Constantes ---
CSV_FILE_PATH = None
CHUNK_SIZE = 100000  # Définir une taille de morceau (par exemple, 100 000 lignes)

def process_csv_by_chunks(path: str, chunk_size: int) -> List[pd.DataFrame] | None:
    """
    Charge un fichier CSV par morceaux, traite chaque morceau,
    et retourne une liste des DataFrames traités.

    Args:
        path (str): Le chemin vers le fichier CSV.
        chunk_size (int): Le nombre de lignes à lire à la fois.

    Returns:
        List[pd.DataFrame] | None: Une liste des DataFrames traités, ou None en cas d'erreur.
    """

    print(f"Tentative de chargement du fichier : {os.path.abspath(path)}")
    print(f"Chargement par morceaux de taille : {chunk_size} lignes.")

    if not os.path.exists(path):
        print(f"Erreur: Le fichier '{path}' est introuvable. Vérifiez le chemin d'accès.")
        return None

    processed_chunks = []
    chunk_index = 0

    try:
        # Créer un itérateur (TextFileReader) au lieu d'un DataFrame unique
        csv_iterator = pd.read_csv(path, chunksize=chunk_size)

        # Parcourir les morceaux générés par l'itérateur
        for chunk in csv_iterator:
            chunk_index += 1
            print(f"Traitement du morceau #{chunk_index} (taille: {len(chunk)} lignes)...")

            # 💡 --- Zone de Traitement des Données --- 💡
            # Ici, vous pouvez appliquer des opérations qui réduisent la taille du morceau,
            # comme le filtrage, l'agrégation ou le calcul de statistiques.

            # Exemple : Calculer la moyenne de toutes les colonnes et stocker
            # stats_df = chunk.mean().to_frame().T
            # processed_chunks.append(stats_df)

            # Exemple : Filtrer pour garder uniquement les lignes où 'col_A' > 10
            # filtered_chunk = chunk[chunk['col_A'] > 10]
            # processed_chunks.append(filtered_chunk)

            # --- Fin de la Zone de Traitement ---

            # Dans cet exemple, nous stockons le morceau complet filtré
            # Si vous avez 500Mo, vous DEVEZ faire un traitement pour réduire le morceau avant de l'ajouter
            # à 'processed_chunks', sinon vous resaturerez votre RAM.
            processed_chunks.append(chunk)


        print(f"\nChargement et traitement terminés. {chunk_index} morceaux traités.")

        # ⚠️ ATTENTION : La ligne suivante va CONSOLIDER TOUS les morceaux.
        # Si vous n'avez pas réduit la taille des morceaux, vous risquez une saturation RAM.
        # Si vous n'avez besoin que de statistiques, vous pouvez retourner processed_chunks directement.
        final_dataframe = pd.concat(processed_chunks, ignore_index=True)
        print(f"Taille du DataFrame final: {len(final_dataframe)} lignes.")
        return final_dataframe

    except Exception as e:
        print(f"Une erreur s'est produite lors du traitement du fichier CSV : {e}")
        return None

In [ ]:
import pandas as pd
import os
from pathlib import Path

# --- PARAMÈTRES GLOBAUX ---
CHUNK_SIZE = 100000  # Taille de chaque bloc de lecture (ex: 100 000 lignes)
SAMPLING_STEP = 10    # Garder 1 ligne sur 10
OUTPUT_DIR = '../raw_data/sampled_data_1_10/' # Répertoire de destination pour les fichiers échantillonnés

# Assurez-vous que le répertoire de sortie existe
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)


def process_csv_by_chunks(path: str, chunk_size: int, sampling_step: int = SAMPLING_STEP) -> None:
    """
    Charge un grand fichier CSV par blocs, échantillonne chaque bloc,
    puis concatène et sauvegarde le DataFrame final échantillonné.
    """
    file_path = Path(path)

    # --- VÉRIFICATION ---
    if not file_path.is_file():
        print(f"🚨 ERREUR : Le fichier spécifié n'existe pas : {path}")
        return

    print(f"\n=======================================================")
    print(f"📖 Démarrage du chargement pour : {file_path.name}")
    print(f"   (Chunk size: {chunk_size}, Sampling: 1/{sampling_step})...")
    print(f"=======================================================")

    # Initialise une liste pour stocker les DataFrames échantillonnés
    sampled_chunks = []
    total_rows_processed = 0

    # Utilisez un itérateur pour lire le fichier CSV par blocs
    try:
        csv_iterator = pd.read_csv(file_path, chunksize=chunk_size)
    except Exception as e:
        print(f"🚨 ERREUR lors de la lecture du CSV : {e}")
        return

    # --- BOUCLE DE TRAITEMENT ---
    for i, chunk in enumerate(csv_iterator):
        print(f"-> Traitement du bloc n°{i+1} ({len(chunk)} lignes)...")

        # 1. Échantillonnage : Garder 1 ligne sur SAMPLING_STEP
        # df.iloc[::step, :] est la syntaxe correcte pour l'échantillonnage des lignes.
        sampled_chunk = chunk.iloc[::sampling_step, :]

        # 2. Ajout du bloc échantillonné à la liste
        sampled_chunks.append(sampled_chunk)

        total_rows_processed += len(chunk)

    print("✅ Fin du traitement des blocs.")

    # --- CONCATÉNATION FINALE ---
    # Concatène tous les DataFrames échantillonnés en un seul DataFrame final
    final_df = pd.concat(sampled_chunks, ignore_index=True)

    # --- SAUVEGARDE DU RÉSULTAT ---

    # Création du nouveau nom de fichier (ex: TEP_FaultFree_Testing_sampled.csv)
    new_file_name = f"{file_path.stem}_sampled{file_path.suffix}"
    output_path = Path(OUTPUT_DIR) / new_file_name

    final_df.to_csv(output_path, index=False) # index=False pour ne pas sauvegarder l'index du DataFrame

    # --- RÉSULTAT ---
    print("\n--- RÉSULTAT FINAL ---")
    print(f"Total des lignes traitées : {total_rows_processed}")
    print(f"Lignes dans le DataFrame final (échantillonné) : {len(final_df)}")
    print(f"Fichier sauvegardé sous : {output_path}")
    print("----------------------")


# --- EXÉCUTION DU CODE ---
pathlist = [
    # '/home/bapt/code/Monitor-the-Reactor/raw_data/TEP_FaultFree_Testing.csv',
    # '/home/bapt/code/Monitor-the-Reactor/raw_data/TEP_FaultFree_Training.csv',
    # '/home/bapt/code/Monitor-the-Reactor/raw_data/TEP_Faulty_Testing.csv',
    '/home/bapt/code/Monitor-the-Reactor/raw_data/TEP_Faulty_Training.csv'
]

# Lance le traitemµent pour chaque fichier de la liste
for path in pathlist:
    process_csv_by_chunks(path, chunk_size=CHUNK_SIZE)


📖 Démarrage du chargement pour : TEP_Faulty_Training.csv
   (Chunk size: 100000, Sampling: 1/10)...
-> Traitement du bloc n°1 (100000 lignes)...
-> Traitement du bloc n°2 (100000 lignes)...
-> Traitement du bloc n°3 (100000 lignes)...
-> Traitement du bloc n°4 (100000 lignes)...
-> Traitement du bloc n°5 (100000 lignes)...
-> Traitement du bloc n°6 (100000 lignes)...
-> Traitement du bloc n°7 (100000 lignes)...
-> Traitement du bloc n°8 (100000 lignes)...
-> Traitement du bloc n°9 (100000 lignes)...
-> Traitement du bloc n°10 (100000 lignes)...
-> Traitement du bloc n°11 (100000 lignes)...
-> Traitement du bloc n°12 (100000 lignes)...
-> Traitement du bloc n°13 (100000 lignes)...
-> Traitement du bloc n°14 (100000 lignes)...
-> Traitement du bloc n°15 (100000 lignes)...
-> Traitement du bloc n°16 (100000 lignes)...
-> Traitement du bloc n°17 (100000 lignes)...
-> Traitement du bloc n°18 (100000 lignes)...
-> Traitement du bloc n°19 (100000 lignes)...
-> Traitement du bloc n°20 (100000